In [6]:
from selenium import webdriver
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [7]:
# 브라우저 열기
driver = webdriver.Chrome()

# 페이지 접속하기
url = 'https://kovo.co.kr/KOVO/game/v-league?first=%ED%8C%80+%EC%88%9C%EC%9C%84'
driver.get(url)

# 명시적 대기 설정 (최대 10초 대기)
wait = WebDriverWait(driver, 10)

# 여자부 클릭하기 (클릭이 가능할 때까지 대기)
try:
    # 요소가 클릭 가능한 상태일 때까지 대기
    click_1 = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label:nth-child(2)")))
    click_1.click()
    time.sleep(1)
except Exception as e:
    print("오류 발생:", e)

In [16]:
for num in range(3):
    # 시즌 클릭하기
    click_2 = driver.find_elements(By.CSS_SELECTOR,"span.ant-select-selection-item")
    click_2[0].click()
    time.sleep(1)
    
    # ActionChains 객체 생성
    actions = ActionChains(driver)
    
    # 방향키 입력 (예: 아래 방향키)
    actions.send_keys(Keys.ARROW_DOWN).perform()
    time.sleep(1)
    
    # 엔터키 입력
    actions.send_keys(Keys.ENTER).perform()
    time.sleep(1)
    
    # HTML 다운 받기 & 해석하기
    soup = BeautifulSoup(driver.page_source, 'lxml')
    time.sleep(1)
    
    # 셀 데이터 크롤링
    stat_list = soup.select("tr > td")
    
    stats = []
    for i in stat_list:
        stat = i.text
        data = [stat]
        stats.append(data)
    
    # ".\xa0"값 제거
    cleaned_stats = [item for item in stats if item != ['\xa0']]
    
    # 리스트를 하나로 합치기
    flattened_list = [item for sublist in cleaned_stats for item in sublist]
    list = flattened_list[:84]
    
    # 리스트를 NumPy 배열로 변환
    np_array = np.array(list)
    
    # reshape 사용
    reshaped_array = np_array.reshape(7, -1)
    
    # 데이터프레임으로 전환
    df = pd.DataFrame(reshaped_array)
    df.columns = ["순위", "팀", "승점", "승", "패", "득세트", "실세트", "세트득실률", "득점",	"실점", "점수득실률", "연속"]
    
    # globals()를 사용하여 df0, df1, df2와 같은 변수명을 동적으로 생성
    globals()[f"table_df{num}"] = df
    
    
    click_3 = driver.find_elements(By.CSS_SELECTOR, "button.font-normal")
    c_num = [1, 8, 9]
    
    for numb in range(3):
        for j in c_num:
            click_3[j].click()
            time.sleep(1)
            
            # HTML 다운 받기 & 해석하기
            soup = BeautifulSoup(driver.page_source, 'lxml')
            time.sleep(1)
            
            # 셀 데이터 크롤링
            stat_list = soup.select("tr > td")
            
            stats = []
            for item in stat_list:
                stat = item.text
                data = [stat]
                stats.append(data)
        
            # ".\xa0"값 제거
            cleaned_stats = [item for item in stats if item != ['\xa0']]
        
            # 리스트를 하나로 합치기
            flattened_list = [item for sublist in cleaned_stats for item in sublist]
            list = flattened_list[84:]
        
            # 리스트를 NumPy 배열로 변환
            np_array = np.array(list)
            
            # reshape 사용
            reshaped_array = np_array.reshape(7, -1)
        
            # 데이터프레임으로 전환
            df = pd.DataFrame(reshaped_array)
        
            # globals()를 사용하여 df1, df8, df9와 같은 변수명을 동적으로 생성
            globals()[f"df{j}"] = df
        
        df1.columns = ["공격_순위", "팀", "경기수", "세트수", "공격_시도", "공격_성공", "공격차단", "공격_범실", "공격_성공률"]
        df8.columns =["서브_순위", "팀", "경기수", "세트수", "서브_시도", "서브_성공", "서브_범실", "서브_세트당평균"]
        df9.columns = ["블로킹_순위", "팀", "경기수", "세트수", "블로킹_시도", "블로킹_성공", "블로킹_유효블락", "블로킹_실패", "블로킹_범실", "블로킹_어시스트", "블로킹_세트당평균"]
        df8 = df8.drop(["경기수", "세트수"], axis = 1)
        df9 = df9.drop(["경기수", "세트수"], axis = 1)
        # df1, df8, df9가 이미 "팀" 기준으로 합쳐질 준비가 되어 있으므로, 각 데이터프레임에서 "팀" 열을 기준으로 병합합니다.
        
        # df1과 df8을 팀 기준으로 병합
        merged_df_1_8 = pd.merge(df1, df8, on="팀", how="outer")
        
        # 그 결과와 df9를 팀 기준으로 병합
        season_df = pd.merge(merged_df_1_8, df9, on="팀", how="outer")
        
        # globals()를 사용하여 season_df0, season_df1, season_df2와 같은 변수명을 동적으로 생성
        globals()[f"season_df{num}"] = season_df
        

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=123.0.6312.106)
Stacktrace:
	GetHandleVerifier [0x00007FF6F0C47032+63090]
	(No symbol) [0x00007FF6F0BB2C82]
	(No symbol) [0x00007FF6F0A4EC65]
	(No symbol) [0x00007FF6F0A2CA7C]
	(No symbol) [0x00007FF6F0ABD687]
	(No symbol) [0x00007FF6F0AD2AC1]
	(No symbol) [0x00007FF6F0AB6D83]
	(No symbol) [0x00007FF6F0A883A8]
	(No symbol) [0x00007FF6F0A89441]
	GetHandleVerifier [0x00007FF6F10425AD+4238317]
	GetHandleVerifier [0x00007FF6F107F70D+4488525]
	GetHandleVerifier [0x00007FF6F10779EF+4456495]
	GetHandleVerifier [0x00007FF6F0D20576+953270]
	(No symbol) [0x00007FF6F0BBE54F]
	(No symbol) [0x00007FF6F0BB9224]
	(No symbol) [0x00007FF6F0BB935B]
	(No symbol) [0x00007FF6F0BA9B94]
	BaseThreadInitThunk [0x00007FFB2D58259D+29]
	RtlUserThreadStart [0x00007FFB2EC2AF38+40]


In [12]:
print(df1.shape)  # df1의 형태 확인
print(df8.shape)  # df8의 형태 확인
print(df9.shape)  # df9의 형태 확인

(7, 9)
(7, 6)
(7, 9)


In [25]:
for num in range(4):
    # 시즌 클릭하기
    click_2 = driver.find_elements(By.CSS_SELECTOR,"span.ant-select-selection-item")
    click_2[0].click()
    time.sleep(1)
    
    # ActionChains 객체 생성
    actions = ActionChains(driver)
    
    # 방향키 입력 (예: 아래 방향키)
    actions.send_keys(Keys.ARROW_DOWN).perform()
    time.sleep(1)
    
    # 엔터키 입력
    actions.send_keys(Keys.ENTER).perform()
    time.sleep(1)
    
    # HTML 다운 받기 & 해석하기
    soup = BeautifulSoup(driver.page_source, 'lxml')
    time.sleep(1)
    
    # 셀 데이터 크롤링
    stat_list = soup.select("tr > td")
    
    stats = []
    for i in stat_list:
        stat = i.text
        data = [stat]
        stats.append(data)
    
    # ".\xa0"값 제거
    cleaned_stats = [item for item in stats if item != ['\xa0']]
    
    # 리스트를 하나로 합치기
    flattened_list = [item for sublist in cleaned_stats for item in sublist]
    list = flattened_list[:72]
    
    # 리스트를 NumPy 배열로 변환
    np_array = np.array(list)
    
    # reshape 사용
    reshaped_array = np_array.reshape(6, -1)
    
    # 데이터프레임으로 전환
    df = pd.DataFrame(reshaped_array)
    df.columns = ["순위", "팀", "승점", "승", "패", "득세트", "실세트", "세트득실률", "득점",	"실점", "점수득실률", "연속"]
    
    # globals()를 사용하여 df0, df1, df2와 같은 변수명을 동적으로 생성
    globals()[f"table_df{num + 3}"] = df
    
    
    click_3 = driver.find_elements(By.CSS_SELECTOR, "button.font-normal")
    c_num = [1, 8, 9]
    
    for numb in range(3):
        for j in c_num:
            click_3[j].click()
            time.sleep(1)
            
            # HTML 다운 받기 & 해석하기
            soup = BeautifulSoup(driver.page_source, 'lxml')
            time.sleep(1)
            
            # 셀 데이터 크롤링
            stat_list = soup.select("tr > td")
            
            stats = []
            for item in stat_list:
                stat = item.text
                data = [stat]
                stats.append(data)
        
            # ".\xa0"값 제거
            cleaned_stats = [item for item in stats if item != ['\xa0']]
        
            # 리스트를 하나로 합치기
            flattened_list = [item for sublist in cleaned_stats for item in sublist]
            list = flattened_list[72:]
        
            # 리스트를 NumPy 배열로 변환
            np_array = np.array(list)
            
            # reshape 사용
            reshaped_array = np_array.reshape(6, -1)
        
            # 데이터프레임으로 전환
            df = pd.DataFrame(reshaped_array)
        
            # globals()를 사용하여 df1, df8, df9와 같은 변수명을 동적으로 생성
            globals()[f"df{j}"] = df
        
        df1.columns = ["공격_순위", "팀", "경기수", "세트수", "공격_시도", "공격_성공", "공격차단", "공격_범실", "공격_성공률"]
        df8.columns =["서브_순위", "팀", "경기수", "세트수", "서브_시도", "서브_성공", "서브_범실", "서브_세트당평균"]
        df9.columns = ["블로킹_순위", "팀", "경기수", "세트수", "블로킹_시도", "블로킹_성공", "블로킹_유효블락", "블로킹_실패", "블로킹_범실", "블로킹_어시스트", "블로킹_세트당평균"]
        df8 = df8.drop(["경기수", "세트수"], axis = 1)
        df9 = df9.drop(["경기수", "세트수"], axis = 1)
        # df1, df8, df9가 이미 "팀" 기준으로 합쳐질 준비가 되어 있으므로, 각 데이터프레임에서 "팀" 열을 기준으로 병합합니다.
        
        # df1과 df8을 팀 기준으로 병합
        merged_df_1_8 = pd.merge(df1, df8, on="팀", how="outer")
        
        # 그 결과와 df9를 팀 기준으로 병합
        season_df = pd.merge(merged_df_1_8, df9, on="팀", how="outer")
        
        # globals()를 사용하여 season_df3, season_df4, season_df5, season_df6와 같은 변수명을 동적으로 생성
        globals()[f"season_df{num + 3}"] = season_df

In [34]:
# num 값에 따라 df와 season_df 병합
for num in range(7):  # 0 ~ 6 번호 순차적으로 처리
    # table_df와 season_df를 팀 기준으로 병합
    merged_df = pd.merge(globals()[f"table_df{num}"], globals()[f"season_df{num}"], on="팀", how="outer")
    # globals()를 사용하여 total_df0, total_df1, total_df2와 같은 변수명을 동적으로 생성
    globals()[f"total_df{num}"] = merged_df

# 인삼공사 -> 정관장 팀명 수정
for i in range(1, 7):
    df_name = f"total_df{i}"  # total_df1, total_df2, ..., total_df6
    globals()[df_name].replace('KGC인삼공사', '정관장', inplace=True)

# 순위로 정렬
for i in range(7):  # 0부터 6까지 반복
    df_name = f"total_df{i}"  # total_df0, total_df1, ..., total_df6
    globals()[df_name] = globals()[df_name].sort_values(by="순위", ascending=True)

# 시즌 컬럼 생성
total_df0["시즌"] = "23-24"
total_df1["시즌"] = "22-23"
total_df2["시즌"] = "21-22"
total_df3["시즌"] = "20-21"
total_df4["시즌"] = "19-20"
total_df5["시즌"] = "18-19"
total_df6["시즌"] = "17-18"

# 세로로 합치기
final_df = pd.concat([total_df0, total_df1, total_df2, total_df3, total_df4, total_df5, total_df6], axis=0, ignore_index=True)

# 컬럼 순서변경
final_df.insert(0, '시즌', final_df.pop('시즌'))

In [35]:
final_df

,시즌,순위,팀,승점,승,패,득세트,실세트,세트득실률,득점,...,서브_범실,서브_세트당평균,블로킹_순위,블로킹_시도,블로킹_성공,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_어시스트,블로킹_세트당평균
0,23-24,1,현대건설,80,26,10,90,51,1.765,3173,...,227,0.96,1,2462,337,934,788,28,269,2.39
1,23-24,2,흥국생명,79,28,8,93,47,1.979,3245,...,243,1.19,3,2422,299,1014,690,37,219,2.14
2,23-24,3,정관장,61,20,16,74,63,1.175,3083,...,284,1,4,2212,292,874,687,30,220,2.13
3,23-24,4,GS칼텍스,51,18,18,63,69,0.913,2914,...,261,1.27,7,1913,221,712,668,33,177,1.67
4,23-24,5,IBK기업은행,51,17,19,70,71,0.986,3114,...,238,0.89,2,2512,330,1024,728,36,274,2.34
5,23-24,6,한국도로공사,39,12,24,58,85,0.682,3034,...,276,0.88,6,2295,278,807,819,45,228,1.94
6,23-24,7,페퍼저축은행,17,5,31,39,101,0.386,2840,...,302,0.72,5,2350,288,915,749,32,254,2.06
7,22-23,1,흥국생명,82,27,9,93,44,2.114,3201,...,288,1.06,6,2432,282,985,728,28,235,2.06
8,22-23,2,현대건설,70,24,12,87,56,1.554,3238,...,289,0.98,4,2541,345,1018,730,22,268,2.41
9,22-23,3,한국도로공사,60,20,16,77,67,1.149,3216,...,233,0.78,1,2628,406,912,807,50,329,2.82


In [37]:
final_df.to_csv('팀별 데이터.csv', index=False)